In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [5]:
engine = create_engine("sqlite:///data/hawaii.sqlite")

In [6]:
# reflect an existing database into a new model using automap_base()
Base = automap_base()
# reflect the tables with Base.prepare(), passing in the engine and reflect=True
Base.prepare(engine, reflect=True)

In [7]:
# We can view all of the classes that automap found with Base.classes
Base.classes.keys()

['measurement', 'station']

In [8]:
# Save references to each table
# Assign the measurement class to a variable called `Measurement`
Measurement = Base.classes.measurement

# Assign the station class to a variable called `Station`
Station = Base.classes.station

In [10]:
# Create our Session() and bind it to the engine
session = Session(engine)

In [12]:
measurement_row = session.query(Measurement).first()
measurement_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x2a386fcc288>,
 'prcp': 0.08,
 'id': 1,
 'date': '2010-01-01',
 'tobs': 65.0,
 'station': 'USC00519397'}

In [18]:
# Use the session to query Measurement table and display the first 15 records
for row in session.query(Measurement.id,Measurement.date, Measurement.tobs, Measurement.prcp).limit(15).all():
    print(row)

(1, '2010-01-01', 65.0, 0.08)
(2, '2010-01-02', 63.0, 0.0)
(3, '2010-01-03', 74.0, 0.0)
(4, '2010-01-04', 76.0, 0.0)
(5, '2010-01-06', 73.0, None)
(6, '2010-01-07', 70.0, 0.06)
(7, '2010-01-08', 64.0, 0.0)
(8, '2010-01-09', 68.0, 0.0)
(9, '2010-01-10', 73.0, 0.0)
(10, '2010-01-11', 64.0, 0.01)
(11, '2010-01-12', 61.0, 0.0)
(12, '2010-01-14', 66.0, 0.0)
(13, '2010-01-15', 65.0, 0.0)
(14, '2010-01-16', 68.0, 0.0)
(15, '2010-01-17', 64.0, 0.0)


In [15]:
station_row = session.query(Station).first()
station_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x2a386f85888>,
 'longitude': -157.8168,
 'id': 1,
 'name': 'WAIKIKI 717.2, HI US',
 'latitude': 21.2716,
 'station': 'USC00519397',
 'elevation': 3.0}

In [19]:
# Use the session to query Station table and display the first 15 records
for station_row in session.query(Station.id, Station.name, Station.latitude, Station.longitude, Station.elevation).limit(15).all():
    print(station_row)

(1, 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0)
(2, 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6)
(3, 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0)
(4, 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9)
(5, 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)
(6, 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5)
(7, 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999998, 32.9)
(8, 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9)
(9, 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)


# Exploratory Climate Analysis

##### Design a query to retrieve the last 12 months of precipitation data

In [ ]:
# Calculate the date 1 year ago from the last data point in the database
# Use the datetime.timedelta() function to help calculating the difference of one year, 12 months, or 365 days
one_yearago = dt.date(2017, 8, 23) - dt.timedelta(days=365)


In [26]:
# Calculate the date 1 year ago from the last data point in the database
max_date = session.query(Measurement).group_by(Measurement.date).max()
print(max_date)

AttributeError: 'Query' object has no attribute 'max'

In [ ]:
# Use the datetime.timedelta() function to help calculating the difference of one year, 12 months, or 365 days
prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)


In [ ]:
# Use session.query() to retrieve the date and prcp columns, .filter() by the date you calculated above, and selecting .all() results


In [ ]:
# Save the query results as a Pandas DataFrame() and set the index to the date column


In [ ]:
# Sort the dataframe by date

# Use Pandas Plotting with Matplotlib to plot the data

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data


##### Design a query to show how many stations are available in this dataset


In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order. Hint: session.query() the station column in the Station table


In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station


In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return (session.query(func.min(Measurement.tobs), 
                         func.avg(Measurement.tobs), 
                         func.max(Measurement.tobs))
                    .filter(Measurement.date >= start_date)
                    .filter(Measurement.date <= end_date)
                    .all())

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    return (session.query(func.min(Measurement.tobs), 
                          func.avg(Measurement.tobs), 
                          func.max(Measurement.tobs))
                   .filter(func.strftime("%m-%d", Measurement.date) == date)
                   .all())
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
